In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
def simulate_pre_post_data(
    n_days_pre=8*7,
    n_days_post=8*7,
    n_total_per_day=14_000,
    p1_pre=0.50,
    ratio_pre=1.10,
    ratio_multiplier_post=0.99,  # set to 1.0 for no effect
    seed=42
):
    """
    Simulate aggregated binomial data for two groups before and after treatment.
    """
    rng = np.random.default_rng(seed)

    days = list(range(-n_days_pre, 0)) + list(range(1, n_days_post + 1))
    rows = []

    # Baseline probabilities
    p2_pre = p1_pre * ratio_pre
    ratio_post = ratio_pre * ratio_multiplier_post
    p1_post = p1_pre
    p2_post = p1_post * ratio_post

    for day in days:
        post = int(day > 0)

        # Choose probabilities
        if post:
            p1, p2 = p1_post, p2_post
        else:
            p1, p2 = p1_pre, p2_pre

        # Group sizes
        n1 = int(0.10 * n_total_per_day)
        n2 = n_total_per_day - n1

        # Simulate successes
        y1 = rng.binomial(n1, p1)
        y2 = rng.binomial(n2, p2)

        rows.append((day, 0, post, y1, n1))
        rows.append((day, 1, post, y2, n2))

    return pd.DataFrame(
        rows,
        columns=["day", "group", "post", "successes", "trials"]
    )


In [ ]:
df_change = simulate_pre_post_data(ratio_multiplier_post=0.99)

In [ ]:
df_change.head()

,day,group,post,successes,trials
0,-56,0,0,682,1400
1,-56,1,0,7032,12600
2,-55,0,0,690,1400
3,-55,1,0,6982,12600
4,-54,0,0,703,1400


In [ ]:
df_change_gb = df_change.groupby(["group", "post"]).sum()
df_change_gb = df_change_gb.reset_index()
df_change_gb

,group,post,day,successes,trials
0,0,0,-1596,39043,78400
1,0,1,1596,39219,78400
2,1,0,-1596,388501,705600
3,1,1,1596,384255,705600


In [ ]:
def run_poisson_log_rr_regression(df):
    """
    Fit Poisson regression with log link and robust SEs.
    """
    df = df.copy()
    df["log_trials"] = np.log(df["trials"])
    df["interaction"] = df["group"] * df["post"]

    X = sm.add_constant(df[["group", "post", "interaction"]])

    model = sm.GLM(
        df["successes"],
        X,
        family=sm.families.Poisson(),
        offset=df["log_trials"]
    )

    results = model.fit(cov_type="HC1")
    return results


In [ ]:
run_poisson_log_rr_regression(df_change).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:              successes   No. Observations:                  224
Model:                            GLM   Df Residuals:                      220
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1120.1
Date:                Sat, 31 Jan 2026   Deviance:                       105.04
Time:                        06:30:36   Pearson chi2:                     105.
No. Iterations:                     6   Pseudo R-squ. (CS):             0.9450
Covariance Type:                  HC1                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.6972      0.003   -239.107      0.000      -0.703      -0.691
group           0.1004      0.003     32.241      0.000       0.094       0.107
post            0.0045      0.005      0.947      0.344      -0.005       0.014
interaction    -0.0155      0.005     -3.089      0.002      -0.025      -0.006
===============================================================================
"""

In [ ]:
# Based on input, should output 0.5, 1.1, 1.0, 0.99
np.exp(-0.6972), np.exp(0.1004), np.exp(0.0045), np.exp(-0.0155)

(np.float64(0.4979776910745289),
 np.float64(1.105613074868341),
 np.float64(1.0045101402046013),
 np.float64(0.9846195067517329))

In [ ]:
run_poisson_log_rr_regression(df_change_gb).summary()

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/_tools.py:121: RuntimeWarning: divide by zero encountered in scalar divide
  scale = np.dot(wresid, wresid) / df_resid
/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:              successes   No. Observations:                    4
Model:                            GLM   Df Residuals:                        0
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -27.115
Date:                Sat, 31 Jan 2026   Deviance:                   1.3532e-11
Time:                        06:29:40   Pearson chi2:                 8.48e-26
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:                  HC1                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.6972   3.73e-16  -1.87e+15      0.000      -0.697      -0.697
group           0.1004   3.73e-16   2.69e+14      0.000       0.100       0.100
post            0.0045   3.73e-16   1.21e+13      0.000       0.004       0.004
interaction    -0.0155   5.88e-16  -2.64e+13      0.000      -0.015      -0.015
===============================================================================
"""

In [ ]:
def run_logistic_regression(df):
    df = df.copy()
    df["interaction"] = df["group"] * df["post"]

    X = sm.add_constant(df[["group", "post", "interaction"]])

    # Binomial GLM with aggregated data
    model = sm.GLM(
        df["successes"] / df["trials"],
        X,
        family=sm.families.Binomial(),
        freq_weights=df["trials"]
    )

    res = model.fit()
    return res


In [ ]:
run_logistic_regression(df_change_gb).summary()

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/_tools.py:121: RuntimeWarning: divide by zero encountered in scalar divide
  scale = np.dot(wresid, wresid) / df_resid
/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                    4
Model:                            GLM   Df Residuals:                  1567996
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -7.0465e+05
Date:                Sat, 31 Jan 2026   Deviance:                  -3.1569e-11
Time:                        06:29:40   Pearson chi2:                 3.90e-26
No. Iterations:                     3   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0080      0.007     -1.121      0.262      -0.022       0.006
group           0.2111      0.008     28.022      0.000       0.196       0.226
post            0.0090      0.010      0.889      0.374      -0.011       0.029
interaction    -0.0333      0.011     -3.123      0.002      -0.054      -0.012
===============================================================================
"""

In [ ]:
run_logistic_regression(df_change).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  224
Model:                            GLM   Df Residuals:                  1567996
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -7.0471e+05
Date:                Sat, 31 Jan 2026   Deviance:                       221.70
Time:                        06:29:40   Pearson chi2:                     222.
No. Iterations:                     3   Pseudo R-squ. (CS):             0.9979
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0080      0.007     -1.121      0.262      -0.022       0.006
group           0.2111      0.008     28.022      0.000       0.196       0.226
post            0.0090      0.010      0.889      0.374      -0.011       0.029
interaction    -0.0333      0.011     -3.123      0.002      -0.054      -0.012
===============================================================================
"""